Do any state/chamber combos have different majorities in different ensembles?

In [1]:
from typing import Dict

import os
import pandas as pd
from collections import defaultdict

from rdapy import DISTRICTS_BY_STATE
from rdametrics import states, chambers, ensembles

Load the all-up scores dataframe from disk

In [2]:
scores_path: str = "~/local/beta-ensembles/dataframe/contents/scores_df.parquet"
scores_df = pd.read_parquet(os.path.expanduser(scores_path))
scores_df.head()

,map,estimated_vote_pct,pr_deviation,estimated_seats,fptp_seats,disproportionality,efficiency_gap_wasted_votes,efficiency_gap_statewide,efficiency_gap,seats_bias,...,proportional_coalitions,minority,county_splitting,district_splitting,counties_split,county_splits,splitting,state,chamber,ensemble
0,2500,0.4837,0.0762,11.8678,11,0.0598,0.1075,0.0744,0.0435,0.0299,...,13,68,1.5858,1.5834,35,64,2,FL,congress,A0
1,5000,0.4837,0.0827,11.6842,11,0.0664,0.0973,0.0744,0.0500,0.0415,...,13,75,1.6487,1.6488,38,71,0,FL,congress,A0
2,7500,0.4837,0.0646,12.1917,12,0.0482,0.0673,0.0387,0.0319,0.0232,...,13,70,1.6020,1.6233,39,70,0,FL,congress,A0
3,10000,0.4837,0.0647,12.1873,12,0.0484,0.0745,0.0387,0.0320,0.0271,...,13,71,1.6085,1.5958,33,67,0,FL,congress,A0
4,12500,0.4837,0.0408,12.8575,13,0.0245,0.0399,0.0030,0.0081,0.0042,...,13,74,1.5676,1.5361,34,60,8,FL,congress,A0


In [3]:
ensembles = [e for e in ensembles if e not in ["A1", "A2", "A3", "A4", "Rev*"]]

for xx in states:
    subset_df = scores_df[
        (scores_df["state"] == xx) & (scores_df["chamber"] == "congress") & (scores_df["ensemble"] == "A0")
    ]
    first_row = subset_df.iloc[0]
    Vf: float = subset_df.iloc[0][
        "estimated_vote_pct"
    ]  # Same for all chambers & ensembles

    for chamber in chambers:
        combo: str = f"{xx}/{chamber}"

        n_districts: int = DISTRICTS_BY_STATE[xx][chamber]
        majority: int = n_districts // 2 + 1
        proportional: int = round(Vf * n_districts)

        majorities: Dict[str, int] = defaultdict(int)

        for ensemble in ensembles:
            mean_seats = scores_df[
                (scores_df["state"] == xx)
                & (scores_df["chamber"] == chamber)
                & (scores_df["ensemble"] == ensemble)
            ]["estimated_seats"].mean()

            if mean_seats > majority:
                majorities["more"] += 1
            else:
                majorities["less"] += 1

        print(
            f"{combo:12} -- districts: {n_districts:>3}, majority: {majority:>2}, D vote share: {Vf:>.2%}, proportional: {proportional:>2d}, # more: {majorities["more"]:>2}, # less: {majorities["less"]:>2}"
        )

print("Done.")


FL/congress  -- districts:  28, majority: 15, D vote share: 48.37%, proportional: 14, # more:  0, # less: 11
FL/upper     -- districts:  40, majority: 21, D vote share: 48.37%, proportional: 19, # more:  0, # less: 11
FL/lower     -- districts: 120, majority: 61, D vote share: 48.37%, proportional: 58, # more:  0, # less: 11
IL/congress  -- districts:  17, majority:  9, D vote share: 58.17%, proportional: 10, # more: 11, # less:  0
IL/upper     -- districts:  59, majority: 30, D vote share: 58.17%, proportional: 34, # more: 11, # less:  0
IL/lower     -- districts: 118, majority: 60, D vote share: 58.17%, proportional: 69, # more: 11, # less:  0
MI/congress  -- districts:  13, majority:  7, D vote share: 51.88%, proportional:  7, # more:  0, # less: 11
MI/upper     -- districts:  38, majority: 20, D vote share: 51.88%, proportional: 20, # more:  0, # less: 11
MI/lower     -- districts: 110, majority: 56, D vote share: 51.88%, proportional: 57, # more:  0, # less: 11
NC/congress  -- dis

No … but entire state/chamber combos are anti majoritarian! (See that notebook for details)